# Construction and validation of bioNLP named entity recognition model using deep learning
Problems: the embeddings I trained is worse than the one from google


Attribution: 
    Some of the code are borrowed from the example code from spacy

In [52]:
import random
import pathlib
import cytoolz
import numpy
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import thinc.extra.datasets
from spacy.compat import pickle
import spacy
import pandas as pd
from sklearn import preprocessing
import keras
from sklearn import model_selection
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt

### load in SRA data

In [128]:
inS_dir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'

sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'

srsS=pd.read_pickle(inS_dir)
%time srsS=pd.Series(data=srsS.values,index=pd.MultiIndex.from_arrays([srsS.index.get_level_values(0),
                                                            srsS.index.get_level_values(1).str.lower()]) )
technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)


### load in spacy models

In [54]:
%%time
#nlp=spacy.load('en_vectors_web_lg')
nlp=spacy.load('./wikipedia-pubmed-and-PMC-w2v')

CPU times: user 51.3 s, sys: 6.4 s, total: 57.7 s
Wall time: 57.6 s


### Functions for model compilation and feature extraction

Useful information for understanding the neural network: 

TimeDistributed: https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/

lr stands for: learning rate

In [129]:
def compile_lstm(embeddings, shape, settings):
    model = Sequential()
    model.add(
        Embedding(
            embeddings.shape[0],
            embeddings.shape[1],
            input_length=shape['max_length'],
            trainable=False,
            weights=[embeddings],
            mask_zero=True
        )
    )
    #the same dense layer is first applied extract the most useful info from embedding layers
    #model.add(TimeDistributed(Dense(shape['nr_hidden'], use_bias=False)))
    model.add(Bidirectional(LSTM(shape['nr_hidden'],
                                 recurrent_dropout=settings['dropout'],
                                 dropout=settings['dropout'])))
    model.add(Dense(shape['nr_class'], activation='sigmoid'))
    model.compile(optimizer=Adam(lr=settings['lr']), loss='binary_crossentropy',
		  metrics=['accuracy'])
    return model

def get_features(docs, max_length):
    docs = list(docs)
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            ##rever to word vector
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs

### subset sra data

In [130]:
#max_length=10
max_sample_per_study_n=100


In [131]:
srsWithText=srsS.index.get_level_values(0).unique()

In [132]:
### take only intersection and than randomize the dataframe
tmpDf=technical_meta_data_df.drop_duplicates('Sample')
tmpInterDf=tmpDf[tmpDf.Sample.isin(srsWithText)]
shuffledDf=tmpInterDf.sample(n=tmpInterDf.shape[0],random_state=0
                                            )
technical_meta_data_df_sub=shuffledDf.groupby('Study').head(n=max_sample_per_study_n)

In [133]:
print ('# of unique studies:',technical_meta_data_df_sub.Study.nunique())

# of unique studies: 106110


### subset data based on entity types 

In [134]:
train_test_ratio=0.8

In [259]:
subsampledSrs=technical_meta_data_df.groupby('Study').head(n=max_sample_per_study_n)['Sample']
#,'dev_stage','phenotype'
myAttribs=['organism','geo_loc_name','strain','sex','tissue','age','genotype','treatment',
           'breed','body_product',
           'disease','cell_line','ethnicity']
#myAttribs=['SCIENTIFIC_NAME','biomaterial_provider','race','population_description','source_name','sex','age','strain','genotype','disease','treatment']
m=srsS.index.get_level_values(0).isin(subsampledSrs.values)
m1=srsS.index.get_level_values(1).isin(myAttribs)
srsS_subS=srsS[m&m1]

In [260]:
myDf=srsS_subS.reset_index()
myDf.columns=['srs','attrib','sentence']

In [261]:
myDf['sentence']=myDf['sentence'].str.replace('_','')

### split data into training and  testing randomly by study levels

It is split by study level to avoid overgeneralization

In [262]:
study_S=technical_meta_data_df_sub['Study'].drop_duplicates()
print (len(study_S))
myNStudies=len(study_S)
train_n=int((myNStudies*train_test_ratio))
train_studies=study_S.sample(n=train_n,random_state=0).values

106110


In [263]:
technical_meta_data_df_sub['Train']=\
    technical_meta_data_df_sub['Study'].isin(train_studies)

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [264]:
train_mask=technical_meta_data_df_sub['Train']
train_samples=technical_meta_data_df_sub['Sample'][train_mask].values
test_samples=technical_meta_data_df_sub['Sample'][~train_mask].values


In [265]:
### set training and testing within the dataframe
#inTrainTestDf=myDf.sample(n=myDf.shape[0]).groupby('attrib').head(n=20000)
cap_size=50000

all_train_df=myDf[myDf.srs.isin(train_samples)]
g=all_train_df.groupby('attrib')
print (g.size().min())
#train_df=g.head(g.size().min())
train_df=all_train_df.sample(n=all_train_df.shape[0]).groupby('attrib').head(n=cap_size)
all_test_df=myDf[myDf.srs.isin(test_samples)]
test_df=all_test_df.sample(n=all_test_df.shape[0]).groupby('attrib').head(cap_size)
#cap_size=g.size().min()




3824


In [266]:
attribVC_train=train_df.attrib.value_counts()#.to_frame()

In [267]:
#attribVC_test.to_frame()

In [268]:
attribVC_train.to_frame()

,attrib
age,50000
sex,50000
treatment,50000
tissue,50000
genotype,50000
strain,50000
geo_loc_name,50000
breed,30322
organism,21191
disease,14996


### set up LSTM model

In [269]:
nr_hidden=64
max_length=65#, # Shape
dropout=0.5
learn_rate=0.001#, # General NN config
nb_epoch=1#
batch_size=100

use the sklearn encoder going back and forth between classes in string format and integer format

In [270]:
le = preprocessing.LabelEncoder()
le.fit(myAttribs)
nr_classes=len(le.classes_)

In [271]:
lstm_shape={'nr_hidden': 64, 'max_length': max_length, 'nr_class': nr_classes}
lstm_settings={'dropout': 0.5, 'lr': 0.001}

In [272]:
embeddings = nlp.vocab.vectors.data
#embedgigs is vocab.vectors.data
model = compile_lstm(embeddings, lstm_shape, lstm_settings)

### transform list of freetexts into a matrix of word embeddings

In [273]:
train_texts=train_df.sentence.tolist()
dev_texts=test_df.sentence.tolist()

train_labels=keras.utils.to_categorical(
    le.transform(train_df.attrib.values))
dev_labels=keras.utils.to_categorical(le.transform(test_df.attrib.values))

train_docs = list(nlp.pipe(train_texts))
dev_docs = list(nlp.pipe(dev_texts))

train_X = get_features(train_docs, lstm_shape['max_length'])
dev_X = get_features(dev_docs, lstm_shape['max_length'])

In [274]:
#train_texts

#### Model training

In [ ]:
lstm=model.fit(train_X, train_labels, validation_data=(dev_X, dev_labels),
          nb_epoch=nb_epoch, batch_size=batch_size)
model.save('./model/lstm.h5')

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 435265 samples, validate on 249125 samples
Epoch 1/1
184500/435265 [===========>..................] - ETA: 5:08 - loss: 0.1403 - acc: 0.9616

In [ ]:
numpy.save('./model/classes.npy', le.classes_)


In [ ]:
val_docs = list(nlp.pipe(dev_texts))
val_X=get_features(val_docs,lstm_shape['max_length'])
predictM=lstm.model.predict_proba(val_X)

In [ ]:
probaDf_multI=pd.DataFrame(data=predictM,columns=le.classes_)

In [ ]:
probaDf_multI.index=pd.MultiIndex.from_arrays([test_df.attrib.values,dev_texts],names=['entity','text'])

In [ ]:
probaDf=probaDf_multI.reset_index()

### show contingency tables

In [ ]:
minProbThreshold=0.2
confidence_mask=probaDf_multI.max(axis=1)>=minProbThreshold
tmpDf3=probaDf_multI[confidence_mask].idxmax(axis=1).reset_index(name='predicted')

In [ ]:
continDf=tmpDf3.groupby(['predicted','entity']).size().unstack().fillna(0)

In [ ]:
#probaDf_multI

In [ ]:
fig,ax=plt.subplots(figsize=(12,5))
#,cmap="Greens"
continDf=continDf.loc[attribVC_train.index,attribVC_train.index]
sns.heatmap(ax=ax,data=continDf/continDf.sum(axis=0),annot=True,cbar_kws={'label':'% of samples'},center=0.3)


#### Performance of model in validation cohort

In [ ]:
fig,ax=plt.subplots()
for myClass in le.classes_: 
    fpr,tpr,_=metrics.roc_curve((probaDf.entity==myClass),probaDf[myClass])    
    ax.plot(fpr,tpr,label="{myClass}  (AUC: {AUC})".format(myClass=myClass,AUC=str(metrics.auc(fpr,tpr))[:5]))
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.legend()

# scratch

In [85]:
asdadasd

NameError: name 'asdadasd' is not defined

### for a sentence split

In [ ]:
import nltk
from nltk.util import ngrams

sent="T cell is treated with LPS grow faster"
s=sent.split(' ')
n_gram=2
grams=list(map(lambda L:" ".join(L),list(ngrams(s,n_gram))))
print (grams)
val_docs = list(nlp.pipe(grams))
val_X=get_features(val_docs,lstm_shape['max_length'])
tmpDf=pd.DataFrame(data=lstm.model.predict_proba(val_X),columns=le.classes_,index=grams)
ax=sns.heatmap(tmpDf,cbar_kws={'label': 'Emitted probability'},annot=True)
ax.set_ylabel('')
ax.set_title('{} grams'.format(n_gram))

In [ ]:
val_X=get_features(val_docs,lstm_shape['max_length'])

In [ ]:
!echo $PWD/Data/DEEP_NLP/NLP_spacy/keras_on_sra_data_v2.ipynb

In [100]:


tmpS3=srsS[srsS.index.get_level_values(1)=='cur_land_use']

In [101]:
len(tmpS3.value_counts())

47

In [102]:
#no data: biomass,chem_mutagen
srsS.head()

ERS069057  TITLE                                Streptococcus pneumoniae
           SCIENTIFIC_NAME                      Streptococcus pneumoniae
           SUBMITTER_ID          SN20648-sc-2011-11-03T19:59:41Z-1101437
           Strain                                                       
           Sample Description                                           
dtype: object